In [1]:
import nltk
from collections import Counter, defaultdict
from nltk.tree import *
from nltk.grammar import *


In [3]:
# optional
nltk.download('large_grammars')


[nltk_data] Downloading package large_grammars to
[nltk_data]     /home/rrichajalota/nltk_data...
[nltk_data]   Unzipping grammars/large_grammars.zip.


True

In [3]:
gramma = nltk.data.load("grammars/large_grammars/atis.cfg") # load the grammar
s = nltk.data.load("grammars/large_grammars/atis_sentences.txt") # load raw sentences
t = nltk.parse.util.extract_test_sentences(s)

In [50]:
def get_key_from_value(my_dict, to_find):
    for k,v in my_dict.items():
        if v==to_find: return k
    return None


mixed = set() # subtract this from the set of all production rules
new_rule = ""
dummy_rules = dict()
remove_rules = set()
all_rules = set()

for prod in gramma.productions():
    if prod.is_nonlexical() and len(prod.rhs()) == 2:
        rhs_nts = [str(sym) for sym in prod.rhs()]
        #print(rhs_nts)
        dummy_rules[str(prod.lhs())] = rhs_nts
        new_rule += str(prod.lhs()) + ' -> '+ ' '.join(rhs_nts) + ' \n'
        remove_rules.add(prod)

for prod in gramma.productions():
    all_rules.add(prod)
    
    if prod.lhs() == gramma.start() and len(prod.rhs()) == 1:
        pp = gramma.productions(lhs=prod.rhs()[0])
        for rule in pp:
            print(rule.rhs())
            new_rule += str(prod.lhs()) + ' -> '+ str(rule.rhs()[0]) + ' \n'
        #new_rule += str(prod.lhs()) + ' -> '+ str(pp[0].rhs()[0]) + ' \n'
        break
    
    # replace such rules so that they point directly to terminals
    if prod.is_nonlexical() and len(prod.rhs()) == 1:
        pp = gramma.productions(lhs=prod.rhs()[0])
        new_rule += str(prod.lhs()) + ' -> "'+ str(pp[0].rhs()[0]) + '" \n'
        remove_rules.add(prod)
    
    elif len(prod.rhs()) > 2:
        rhs_nts = list(prod.rhs()) 
        while len(rhs_nts) != 2:
            # take the 2 leftmost nodes and combine them 
            l1, l2 = rhs_nts[0], rhs_nts[1]
            dum_key = str(l1) + '_' + str(l2)
            if dum_key not in dummy_rules or get_key_from_value(dummy_rules, [str(l1), str(l2)]) is None:
                dummy_rules[dum_key] = [str(l1), str(l2)]
            #else:
                #print(f'{dum_key} already hai')
            rhs_nts = [dum_key] + rhs_nts[2:]
            
        rhs_nts = [str(nt) for nt in rhs_nts]
        new_rule += str(prod.lhs()) + ' -> ' + ' '.join(rhs_nts) + '\n'
        remove_rules.add(prod)
        #print(new_rule)
                
#print(new_rule)
for k, v in dummy_rules.items(): #rules that have been added to dict
    new_rule += k + ' -> ' + ' '.join(v) + '\n'
#gram = nltk.CFG.fromstring(new_rule)


(NP_WPS, VERB_VBZ, INFCL_VB)
(NP_NN, VERB_VBZ, PP_NN, AVP_RB, NP_CD)
(NP_NN, NP_WPS, VERB_VBZ, PP_NN)
(AVP_RB, NP_PPS, VERB_VBZ, NP_NN)
(NP_CD, VERB_VBZ)


In [5]:
new_gram_set = all_rules - remove_rules
len(new_gram_set)

925

In [6]:
for rule in new_gram_set:
    new_rule += str(rule) + '\n'

gram = nltk.CFG.fromstring(new_rule)
gram.is_chomsky_normal_form()


True

In [7]:
print(len(new_gram_set), len(all_rules), len(remove_rules))
print(len(gram.productions()))

925 5517 4592
9036


In [12]:
s = gramma.productions(lhs=Nonterminal('SIGMA'))
for r in s:
    print(len(r.rhs()))
    print(gramma.productions(lhs=r.rhs()[0]))
    break

1
[NREL_VBZ -> NP_WPS VERB_VBZ INFCL_VB, NREL_VBZ -> NP_NN VERB_VBZ PP_NN AVP_RB NP_CD, NREL_VBZ -> NP_NN NP_WPS VERB_VBZ PP_NN, NREL_VBZ -> AVP_RB NP_PPS VERB_VBZ NP_NN, NREL_VBZ -> NP_CD VERB_VBZ]


In [85]:
with open('generated_cnf_grammar.txt', 'w') as f:
    for rule in new_rule.split('\n'):
        print(rule)
        f.write(rule+'\n')
        

SIGMA -> ADJ_ABL JYL 
GCC -> AVP_RB GCD 
GCD -> NP_DTS pt_char_per 
GCA -> AVP_RB GCB 
GCB -> COMPCL_VBZ pt_char_per 
GCK -> NP_NNS GCL 
GCL -> AJP_JJ pt_char_per 
GCI -> NP_DTS GCJ 
GCJ -> AJP_JJR pt_char_per 
GCG -> NP_DT GCH 
GCH -> AJP_JJ pt_char_per 
GCE -> AVP_RB GCF 
GCF -> NP_NNS pt_char_per 
GCT -> NP_NN pt_char_per 
GCS -> VERB_BEZ GCT 
GCR -> NP_NNS pt_char_per 
GCQ -> NP_PPSS GCR 
GCP -> NP_NN pt_char_per 
GCO -> NP_PPSS GCP 
GCN -> AJP_JJR pt_char_per 
GCM -> NP_NNS GCN 
GCZ -> NP_NN pt_char_per 
GCY -> VERB_BEZ GCZ 
GCX -> NP_NN pt_char_per 
GCW -> VERB_BEZ GCX 
GCV -> NP_NP pt_char_per 
GCU -> VERB_BEZ GCV 
GDB -> AJP_JJ pt_char_per 
GDC -> VERB_BEZ GDD 
GDD -> AJP_JJT pt_char_per 
GDE -> VERB_BEZ GDF 
GDA -> VERB_BEZ GDB 
GDJ -> COMPCL_VB pt_char_per 
GDK -> VERB_BEZ GDL 
GDL -> NP_CC pt_char_per 
GDM -> VERB_BEZ GDN 
GDF -> NP_CC pt_char_per 
GDG -> VERB_BEZ GDH 
GDH -> NP_NN pt_char_per 
GDI -> VERB_BEZ GDJ 
GDS -> VERB_BEZ GDT 
GDR -> NP_NNS pt_char_per 
GDU -> VERB_

OM -> AVP_RB pt_char_per 
IFY -> NP_CD IFZ 
IFV -> CONJ_CC NP_CD 
ON -> NP_NNS OO 
OO -> VERB_VB OP 
IFW -> NP_CD IFX 
OH -> INFCL_VB pt_char_per 
OI -> NP_PPSS OJ 
IFZ -> CONJ_CC NP_NP 
OJ -> VERB_MD OK 
OK -> VERB_VB OL 
IFP -> VERB_VBZ PP_NN 
OD -> PP_NN OE 
OE -> NP_PPSS OF 
IFQ -> CONJ_CC IFR 
IFN -> VERB_VBZ NP_NN 
OF -> VERB_MD OG 
OG -> VERB_VB OH 
IFO -> NP_WPS IFP 
IFT -> AVP_RB PP_NP 
OA -> VERB_VB OB 
IFU -> NP_CD IFV 
OB -> NP_NN OC 
IFR -> AVP_RB PP_CC 
OC -> PP_NN pt_char_per 
IFS -> CONJ_CC IFT 
GJ -> NP_PPO GK 
GK -> NP_NN GL 
GH -> PP_NP GI 
GI -> PP_NP pt_char_per 
GN -> PP_NP GO 
GO -> PP_NP pt_char_per 
GL -> AVP_RB GM 
GM -> PP_NNS GN 
GR -> PP_NNS GS 
GS -> AVP_RB GT 
GP -> NP_PPO GQ 
GQ -> NP_NN GR 
GV -> NP_PPO GW 
GW -> NP_NNS GX 
GT -> PP_NP GU 
GU -> PP_NP pt_char_per 
GZ -> PP_NN HA 
GY -> PP_NP GZ 
GX -> AJP_JJ GY 
HD -> PP_NNS HE 
HC -> NP_NNS HD 
HB -> NP_PPO HC 
HA -> PP_CD pt_char_per 
HH -> NP_PPO HI 
HG -> PP_NP pt_char_per 
HF -> PP_NP HG 
HE -> PP_

NP_NP -> QUANP_ABN NOUN_NP 
NP_NP -> a c 
NP_NP -> a s 
NP_NP -> air taxi 
NP_NP -> alpha sierra 
NP_NP -> america west 
NP_NP -> american west 
NP_NP -> april eighth 
NP_NP -> april fourth 
NP_NP -> april ninth 
NP_NP -> april sixth 
NP_NP -> b JPK 
NP_NP -> b w 
NP_NP -> bay area 
NP_NP -> boston pt250 
NP_NP -> c o 
NP_NP -> c p 
NP_NP -> charlotte JPL 
NP_NP -> chicago o___hare 
NP_NP -> columbus JPM 
NP_NP -> columbus JPN 
NP_NP -> d c 
NP_NP -> d l 
NP_NP -> dallas JPO 
NP_NP -> december JPP 
NP_NP -> december JPQ 
NP_NP -> december JPR 
NP_NP -> december JPS 
NP_NP -> december JPT 
NP_NP -> december twenty 
NP_NP -> denver dallas 
NP_NP -> detroit JPU 
NP_NP -> e JPV 
NP_NP -> el paso 
NP_NP -> f f 
NP_NP -> february JPW 
NP_NP -> february JPX 
NP_NP -> february JPY 
NP_NP -> fort JPZ 
NP_NP -> fort worth 
NP_NP -> friday JQA 
NP_NP -> h JQB 
NP_NP -> h p 
NP_NP -> i JQC 
NP_NP -> indiana indianapolis 
NP_NP -> indianapolis HCS 
NP_NP -> j JQD 
NP_NP -> john JQE 
NP_NP -> john k

DECL_BER -> NP_NNS AKA 
DECL_BER -> NP_NNS AKE 
DECL_BER -> NP_NNS AKI 
DECL_BER -> NP_NNS AKM 
DECL_BER -> NP_NNS AKQ 
DECL_BER -> NP_NNS AKU 
DECL_BER -> NP_NNS AKY 
DECL_BER -> NP_NNS ALC 
DECL_BER -> NP_NNS ALG 
DECL_BER -> NP_NNS ALK 
DECL_BER -> NP_NNS BH 
DECL_BER -> NP_NNS CRG 
DECL_BER -> NP_NNS CRJ 
DECL_BER -> NP_NNS CRM 
DECL_BER -> NP_NNS CRP 
DECL_BER -> NP_NNS CRS 
DECL_BER -> NP_NNS CRV 
DECL_BER -> NP_NNS CRY 
DECL_BER -> NP_NNS GAS 
DECL_BER -> NP_NNS GAU 
DECL_BER -> NP_NNS GAW 
DECL_BER -> NP_NNS GAY 
DECL_BER -> NP_NNS GBA 
DECL_BER -> NP_NNS GBC 
DECL_BER -> NP_NNS GBE 
DECL_BER -> NP_NNS GBG 
DECL_BER -> NP_NNS GBI 
DECL_BER -> NP_NNS GBK 
DECL_BER -> NP_NNS JN 
DECL_BER -> NP_NNS JS 
DECL_BER -> NP_NNS JX 
DECL_BER -> NP_NNS KC 
DECL_BER -> NP_NP KH 
DECL_BER -> NP_ONS GBM 
DECL_BER -> NP_WPS CSB 
DECL_BER -> NP_WPS GBO 
DECL_BER -> NP_WPS GBQ 
DECL_BER -> NP_WPS GBS 
DECL_BER -> NP_WPS GBU 
DECL_BER -> NP_WPS GBW 
DECL_BER -> PP_NN CSE 
DECL_BER -> PP_NNS CSH 


SIGMA -> VERB_MD BJW 
SIGMA -> VERB_MD BKA 
SIGMA -> VERB_MD BKE 
SIGMA -> VERB_MD BKI 
SIGMA -> VERB_MD BKM 
SIGMA -> VERB_MD BOE 
SIGMA -> VERB_MD BOI 
SIGMA -> VERB_MD BOQ 
SIGMA -> VERB_MD CZU 
SIGMA -> VERB_MD CZX 
SIGMA -> VERB_MD DPJ 
SIGMA -> VERB_MD DPM 
SIGMA -> VERB_MD DPP 
SIGMA -> VERB_MD DPS 
SIGMA -> VERB_MD DPV 
SIGMA -> VERB_MD DUU 
SIGMA -> VERB_MD DUX 
SIGMA -> VERB_MD DXU 
SIGMA -> VERB_MD EB 
SIGMA -> VERB_MD EH 
SIGMA -> VERB_MD EN 
SIGMA -> VERB_MD ET 
SIGMA -> VERB_MD GOU 
SIGMA -> VERB_MD GOW 
SIGMA -> VERB_MD GTS 
SIGMA -> VERB_MD I 
SIGMA -> VERB_MD JIK 
SIGMA -> VERB_MD JJN 
SIGMA -> VERB_MD JJO 
SIGMA -> VERB_MD JJP 
SIGMA -> VERB_MD MA 
SIGMA -> VERB_MD MF 
SIGMA -> VERB_MD RU 
SIGMA -> VERB_MD RZ 
SIGMA -> VERB_MD SE 
SIGMA -> VERB_MD SJ 
SIGMA -> VERB_MD SO 
SIGMA -> VERB_MD ST 
SIGMA -> VERB_MD SY 
SIGMA -> VERB_VB AAB 
SIGMA -> VERB_VB BA 
SIGMA -> VERB_VB BTS 
SIGMA -> VERB_VB BTW 
SIGMA -> VERB_VB BUA 
SIGMA -> VERB_VB BUE 
SIGMA -> VERB_VB BUI 
SIGM

IHE -> NP_NP IHF
IHF -> CONJ_CC NP_VBG
IHG -> NP_NP IHH
IHA -> NP_NP IHB
IHB -> CONJ_CC NP_NP
IHC -> NP_NP IHD
IHL -> CONJ_CC NP_NP
IHM -> NP_NPS IHN
IHN -> CONJ_CC NP_NPS
IHO -> PP_NP IHP
IHH -> CONJ_CC NREL_VBZ
IHI -> NP_NPS IHJ
IHJ -> CONJ_CC NP_NN
IHK -> NP_NPS IHL
AW -> PP_PPO AX
IIT -> AJP_JJ NOUN_NN
AV -> NP_NN AW
IIS -> ADJ_AT IIT
IIR -> AJP_AP NOUN_NN
AY -> PP_NP AZ
AX -> PP_NP AY
IIQ -> ADJ_AT IIR
IIX -> NOUN_NN NAPPOS_NN
AS -> PP_NP pt_char_per
AR -> PP_NP AS
IIW -> ADJ_AT IIX
AU -> VERB_VB AV
IIV -> AJP_JJT NOUN_NN
AT -> NP_PPSS AU
IIU -> ADJ_AT IIV
BB -> NP_NNS BC
IIZ -> NOUN_NN NAPPOS_NP
BA -> NP_PPO BB
IIY -> ADJ_AT IIZ
IJA -> ADJ_AT IJB
AZ -> PP_NN pt_char_per
AF -> NP_PPSS AG
IIC -> AVP_RB IID
AG -> VERB_HV AH
IID -> NOUN_CD PP_NN
AH -> NP_NN AI
IIA -> AVP_RB IIB
IIB -> NOUN_CD AVP_RB
AI -> PP_NP AJ
IIG -> NOUN_CD IIH
AB -> VERB_VBG AC
IIH -> PP_NN RELCL_HV
AC -> NP_PPO AD
AD -> AVP_RB AE
IIE -> AVP_RB IIF
IIF -> NOUN_CD PRPRTCL_VBG
AE -> AVP_RB NP_NN
AN -> NP_PPSS AO


CUK -> NP_NN CUL
CUL -> PP_NP pt_char_per
CUI -> PP_NN pt_char_per
CUJ -> VERB_BEZ CUK
CUG -> VERB_BEZ CUH
CUH -> NP_NN CUI
CUE -> NP_NN CUF
CUF -> PP_CD pt_char_per
CUC -> AVP_RB pt_char_per
CUD -> VERB_BEZ CUE
CUB -> NP_NN CUC
CUA -> VERB_BEZ CUB
CTZ -> AVPNP_CD pt_char_per
CTY -> NP_NN CTZ
CTX -> VERB_BEZ CTY
CTW -> NP_NN pt_char_per
CTV -> VERB_BEZ CTW
CTU -> NP_WPS CTV
CTT -> NP_NN pt_char_per
CTS -> VERB_BEZ CTT
CTR -> NP_DT CTS
CTN -> PP_NN pt_char_per
CTO -> NP_PPSS CTP
CTP -> AJP_JJ CTQ
CTQ -> PP_NP pt_char_per
CTJ -> AJP_JJ CTK
CTK -> AVPNP_NNS pt_char_per
CTL -> NP_NNS CTM
CTM -> AJP_JJ CTN
CTF -> AVP_RB CTG
CTG -> NP_NNS CTH
CTH -> PP_NP pt_char_per
CTI -> NP_NNS CTJ
CTB -> PP_CD pt_char_per
CTC -> AVP_RB CTD
CTD -> NP_NNS CTE
CTE -> PP_NN pt_char_per
CTA -> NP_NNS CTB
CSZ -> AVP_RB CTA
CSY -> AVPNP_NN pt_char_per
CSV -> AJP_JJ pt_char_per
CSU -> NP_NNS CSV
CSX -> NP_NNS CSY
CSW -> AVP_RB CSX
CSR -> NP_NNS CSS
CSQ -> pt_verb_ber CSR
CST -> AVP_RB CSU
CSS -> PP_NP pt_char_pe

KDA -> pt197 NOUN_NNS
ABD -> PP_NN ABE
KCZ -> NP_NP NOUN_NNS
ABG -> PP_NP ABH
KCY -> NP_NNS NOUN_NNS
ABF -> VERB_VB ABG
ABI -> PP_NN ABJ
ABH -> PP_NP ABI
KDG -> NOUN_NNS RELCL_VB
ABK -> a ABL
ABJ -> NP_PPO NP_NNS
KDF -> NOUN_NNS PP_NP
KDI -> NP_NN NOUN_NNS
ABM -> thousand ABN
ABL -> one ABM
KDH -> NOUN_NNS pt_char_per
KDC -> AJP_JJ NOUN_NNS
ABO -> hundred nineteen
ABN -> three ABO
KDB -> ADJ_AP NOUN_NNS
ABQ -> two ABR
KDE -> AJP_JJ NOUN_NNS
KDD -> NOUN_NNS RELCL_VBN
ABP -> thousand ABQ
KCO -> NOUN_NNS PP_CC
AAS -> PP_NPS AAT
AAT -> PP_NP AAU
KCP -> NOUN_NNS PP_CD
KCM -> ADJ_AT NOUN_NNS
AAQ -> VERB_VB AAR
AAR -> NP_NNS AAS
KCN -> AJP_JJ NOUN_NNS
KCK -> NP_NN NOUN_NNS
AAW -> PP_NP AAX
KCL -> NP_NNS NOUN_NNS
AAX -> PP_NP AAY
KCI -> NOUN_NNS PRPRTCL_VBG
AAU -> PP_NN PP_NP
AAV -> VERB_VB AAW
KCJ -> NOUN_NNS pt_char_per
KCW -> NOUN_NNS RELCL_VB
KCX -> NP_NN NOUN_NNS
AAY -> AVPNP_NN AAZ
KCU -> NOUN_NNS PRPRTCL_VBG
AAZ -> AVP_RB NREL_BEZ
KCV -> NOUN_NNS RELCL_CC
KCS -> NOUN_NNS PP_NP
ABA -> VE

AJP_JJ -> 'e'
VERB_VBZ -> 'continues'
NP_NN -> 'going'
ADJ_CD -> 'twelve'
AVPNP_NN -> 'manufacturer'
VERB_VBG -> 'continuing'
VP_VB -> 'serve'
pt_adj_jj -> 'total'
SIGMA -> 'departures'
AVPNP_NNS -> 'aircrafts'
NP_NP -> 'northwest'
AJP_JJ -> 'evening'
NOUN_CD -> 'thirteen'
AVPNP_NNS -> 'flights'
does -> 'does'
a -> 'a'
NOUN_NN -> 'limo'
NAPPOS_NP -> 'virginia'
following -> 'following'
okay -> 'okay'
pt_prep_in -> 'up'
INFCL_VB -> 'decide'
NP_NN -> 'amount'
AVPNP_NP -> 'christmas'
AJP_JJ -> 'red'
SIGMA -> 'use'
AJP_JJ -> 'avoiding'
NOUN_NN -> 'morning'
VERB_VB -> 'carry'
ADJ_DTI -> 'much'
pt_adj_jjt -> 'fastest'
NP_NP -> 'cleveland'
SIGMA -> 'connection'
NAPPOS_NP -> 'c'
VERB_VBG -> 'descending'
NOUN_NPS -> 'illinois'
pt_adj_jjt -> 'busiest'
AVPNP_NP -> 'logan'
SIGMA -> 'zero'
AVPNP_NN -> 'taxi'
AVPNP_NN -> 'air'
VP_VB -> 'leave'
VP_VB -> 'spend'
pt217 -> 'leave'
pt_noun_nps -> 'illinois'
AVPNP_NP -> 'n'
AVP_QL -> 'too'
SIGMA -> 'disneyland'
pt_verb_vb -> 'visit'
NOUN_NN -> 'family'
NOU

In [129]:
# initialize the parser
parser = nltk.parse.BottomUpChartParser(grammar)
# parse all test sentences
for sentence in t:
    pp = parser.parse(sentence[0])
    print(len(list(pp)))
    break

2085


https://www.nltk.org/api/nltk.html#nltk.grammar.CFG
https://www.nltk.org/api/nltk.html#nltk.tree.Tree.pformat_latex_qtree

In [6]:
grammar.is_chomsky_normal_form()

True

In [81]:
for idx, sent in enumerate(test_sents[15:31]):
 print(sent)
 parser.chart_parse(sent[0])


(['what', 'is', 'e', 'w', 'r', '.'], None)
(['oakland', 'to', 'salt', 'lake', 'city', '.'], None)
(['show', 'me', 'northwest', 'flights', 'to', 'detroit', '.'], None)
(['i', 'want', 'to', 'leave', 'before', 'noon', '.'], None)
(['what', 'flights', 'leave', 'boston', 'to', 'pittsburgh', '.'], None)
(['i', "'d", 'like', 'an', 'afternoon', 'flight', '.'], None)
(['are', 'any', 'fares', 'cheaper', 'than', 'these', '.'], None)
(['what', 'is', 'the', 'flying', 'time', 'from', '.'], None)
(['which', 'flights', 'use', 'a', 'large', 'plane', '.'], None)
(['what', 'area', 'does', 'canadian', 'airlines', 'international', 'service', '.'], None)
(['what', 'flights', 'leave', 'las', 'vegas', 'to', 'oakland', '.'], None)


KeyboardInterrupt: 

In [40]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP | P N
NP -> Det N | Det NPP | 'I' | 'John'
NPP -> N PP | N P 
VP -> V NP | VP PP
Det -> 'an' | 'my' | 'a'
N -> 'elephant' | 'pajamas' | 'shot' | 'cake'
V -> 'shot' | 'ate'
P -> 'in'
""")

In [128]:
type(grammar)

nltk.grammar.CFG

In [129]:
grammar.start()

S

In [41]:
grammar.productions()

[S -> NP VP,
 PP -> P NP,
 PP -> P N,
 NP -> Det N,
 NP -> Det NPP,
 NP -> 'I',
 NP -> 'John',
 NPP -> N PP,
 NPP -> N P,
 VP -> V NP,
 VP -> VP PP,
 Det -> 'an',
 Det -> 'my',
 Det -> 'a',
 N -> 'elephant',
 N -> 'pajamas',
 N -> 'shot',
 N -> 'cake',
 V -> 'shot',
 V -> 'ate',
 P -> 'in']

In [92]:
from nltk.grammar import *
gram.productions(lhs=Nonterminal("NP"))

[]

In [7]:
grammar.productions(rhs=Nonterminal("Det"))

[NP -> Det N, NP -> Det NPP]

In [8]:
pp = grammar.productions(rhs="I")
pp

[NP -> 'I']

In [18]:
def fetch_lhs(grammar, rhs_tuple):
    all_prods = []
    
    if len(rhs_tuple) == 1: # it's a terminal
        pp = grammar.productions(rhs=rhs_tuple[0])
        for prod in pp:
            all_prods.append(prod.lhs()) 
        return all_prods
    
    filtered_productions = grammar.productions(rhs=rhs_tuple[0])
    #print(filtered_productions)
    
    for a_production in filtered_productions:  
        if rhs_tuple[1] == a_production.rhs()[1]:  
              all_prods.append(a_production.lhs())
    return all_prods

In [10]:
print(fetch_lhs(('Det','N')))

TypeError: fetch_lhs() missing 1 required positional argument: 'rhs_tuple'

In [48]:
grammar.productions(rhs='shot')[0].lhs()

N

In [46]:
def parse_tree(chart, row, col, sym):
    cell = chart[row][col][sym]
    
    #base condition - if string, we've reached a terminal. returns [terminal]
    if type(cell[0]) == type('str'):
        return [ImmutableTree(sym, cell)]
        #return cell
    
    #print(f'cell: {cell}')
    trees = []
    for ind, tup in enumerate(cell):
        #print(f'choice of tuple: {ind}')
        left, right = tup[0], tup[1]
        lhs_trees = parse_tree(chart, left[0], left[1], left[2])
        rhs_trees = parse_tree(chart, right[0], right[1], right[2])
        #print(f'lhs: {lhs_trees}')
        #print(f'rhs: {rhs_trees}')
        for t1 in lhs_trees:
            for t2 in rhs_trees:
                trees.append(ImmutableTree(sym, [t1]+[t2]))
        #for t in trees:
        #    print(f' after append: {Tree.fromstring(str(t)).pretty_print()}')
    return trees
    

def cky_recognizer(words, grammar, parser=False, draw_tree=False, compute_trees=True):
    #print(parser)
    #words = words.split() # for toy eg. 
    n = len(words)
    print(n)
    chart = [[defaultdict(list) for j in range(n+1)]  for i in range(n)]
    
    for col in range(1, n+1): # left to right
        NT_list = fetch_lhs(grammar, (words[col-1],)) 
        for N in NT_list:
            chart[col-1][col][N].append(words[col-1])
            #print(N, chart[col-1][col][N][0])
        
        for row in range(col-2, -1, -1): # bottom-up
            for k in range(row+1, col):
                for first_N in list(chart[row][k].keys()):
                    for second_N in list(chart[k][col].keys()):
                        #print(f'chart[{row}][{k}] : {chart[row][k]}')
                        #print(f'chart[{k}][{col}] : {chart[k][col]}')
                        NT_list = fetch_lhs(grammar, (first_N, second_N))
                        #print(NT_list)
                        for N in NT_list:
                            chart[row][col][N].append(((row, k, first_N), (k, col, second_N)))
                            #print(N, chart[row][col][N])
                            
          
    if parser: 
        if compute_trees:
            num_trees = 0
            if grammar.start() not in chart[0][n].keys(): return 0
            
            for tup in chart[0][n][grammar.start()]:
                print(tup)
                left, right = tup[0], tup[1]
                num_trees += len(chart[left[0]][left[1]][left[2]]) * len(chart[right[0]][right[1]][right[2]])
            return(num_trees)
        
        elif grammar.start() in chart[0][n].keys():
            #all_trees = []
            tree = parse_tree(chart, 0, n, grammar.start())
            print(f'number of trees: {len(tree)}')
            if draw_tree:
                for t in tree:
                    Tree.fromstring(str(t)).pretty_print()
            return len(tree)
        else:
            return 0
    
    #for k, v in chart[0][n].items():
    #    print(f'{k} -> {len(v)} {v}')
    #    print()
    return grammar.start() in chart[0][n].keys()

In [44]:
cky_recognizer(['b', 'a', 'a', 'b', 'a'], toy_gram, parser=True, compute_trees=True)

5


2

In [23]:
toy_gram = nltk.CFG.fromstring("""
S -> A B
S -> B C
A -> B A
B -> C C
C -> A B
A -> 'a'
B -> 'b'
C -> 'a'
""")

In [21]:
toy_gram.productions()

[S -> A B,
 S -> B C,
 A -> B A,
 B -> C C,
 C -> A B,
 A -> 'a',
 B -> 'b',
 C -> 'a']

In [47]:
cky_recognizer('I shot an elephant in my pajamas'.split(), grammar, True, False, True)

7
((0, 1, NP), (1, 7, VP))


2

In [151]:
fetch_lhs((Nonterminal("NP"), Nonterminal("NP")))

[S -> NP VP]


[]

In [119]:
t = Tree.fromstring('(S (NP I) (VP (V enjoyed) (NP my cookie)))')
print(t)

(S (NP I) (VP (V enjoyed) (NP my cookie)))


In [120]:
t.pretty_print()

             S                
  ___________|___              
 |               VP           
 |      _________|___          
 NP    V             NP       
 |     |          ___|____     
 I  enjoyed      my     cookie



In [37]:
t = ImmutableTree('S', [ImmutableTree('NP', ['I']), ImmutableTree('VP', [ImmutableTree('V', ['ate']), ImmutableTree('NP', ['a cookie'])])])
from nltk.draw.tree import draw_trees
draw_trees(t)                    


In [25]:
## test the parser with atis test sentences 
grammar = nltk.data.load("./atis/atis-grammar-cnf.cfg")
sents = nltk.data.load('./atis/atis-test-sentences.txt')
test_sents = nltk.parse.util.extract_test_sentences(sents)

#print(len(grammar.productions()))

with open('own_gram_result.txt', 'w') as f:
    for idx, sent in enumerate(test_sents):
        print(sent[0])
        f.write(f" {' '.join(sent[0])}\t{cky_recognizer(sent[0], gram, True)}\n")
    #print(parser.chart_parse(sent[0]))


['prices', '.']
2
chart[0][1] : defaultdict(<class 'list'>, {pt207: ['prices', 'prices']})
chart[1][2] : defaultdict(<class 'list'>, {pt_char_per: ['.', '.']})
['show', 'availability', '.']
3
chart[0][1] : defaultdict(<class 'list'>, {NOUN_NN: ['show'], VERB_VB: ['show'], show: ['show', 'show']})
chart[1][2] : defaultdict(<class 'list'>, {pt_noun_nn: ['availability', 'availability']})
chart[0][1] : defaultdict(<class 'list'>, {NOUN_NN: ['show'], VERB_VB: ['show'], show: ['show', 'show']})
chart[1][2] : defaultdict(<class 'list'>, {pt_noun_nn: ['availability', 'availability']})
chart[0][1] : defaultdict(<class 'list'>, {NOUN_NN: ['show'], VERB_VB: ['show'], show: ['show', 'show']})
chart[1][2] : defaultdict(<class 'list'>, {pt_noun_nn: ['availability', 'availability']})
chart[1][2] : defaultdict(<class 'list'>, {pt_noun_nn: ['availability', 'availability']})
chart[2][3] : defaultdict(<class 'list'>, {pt_char_per: ['.', '.']})
['show', 'the', 'flights', '.']
4
chart[0][1] : defaultdict(<

chart[2][3] : defaultdict(<class 'list'>, {PREP_IN: ['like'], VERB_VB: ['like'], like: ['like', 'like']})
chart[3][4] : defaultdict(<class 'list'>, {ADV_RB: ['to'], PREP_IN: ['to'], to: ['to', 'to']})
chart[2][3] : defaultdict(<class 'list'>, {PREP_IN: ['like'], VERB_VB: ['like'], like: ['like', 'like']})
chart[3][4] : defaultdict(<class 'list'>, {ADV_RB: ['to'], PREP_IN: ['to'], to: ['to', 'to']})
chart[2][3] : defaultdict(<class 'list'>, {PREP_IN: ['like'], VERB_VB: ['like'], like: ['like', 'like']})
chart[3][4] : defaultdict(<class 'list'>, {ADV_RB: ['to'], PREP_IN: ['to'], to: ['to', 'to']})
chart[2][3] : defaultdict(<class 'list'>, {PREP_IN: ['like'], VERB_VB: ['like'], like: ['like', 'like']})
chart[3][4] : defaultdict(<class 'list'>, {ADV_RB: ['to'], PREP_IN: ['to'], to: ['to', 'to']})
chart[2][3] : defaultdict(<class 'list'>, {PREP_IN: ['like'], VERB_VB: ['like'], like: ['like', 'like']})
chart[3][4] : defaultdict(<class 'list'>, {ADV_RB: ['to'], PREP_IN: ['to'], to: ['to', 'to

chart[3][6] : defaultdict(<class 'list'>, {NOUN_NN_PP_NN: [((3, 4, NOUN_NN), (4, 6, PP_NN))], NP_NN: [((3, 4, NOUN_NN), (4, 6, PP_NN)), ((3, 4, NOUN_NN), (4, 6, PP_NN)), ((3, 5, NP_NN), (5, 6, NOUN_NN)), ((3, 5, NP_NN), (5, 6, NOUN_NN))], VERB_VB_PP_NN: [((3, 4, VERB_VB), (4, 6, PP_NN))], VP_VB: [((3, 4, VERB_VB), (4, 6, PP_NN)), ((3, 4, VERB_VB), (4, 6, PP_NN))], NP_NN_NOUN_NN: [((3, 5, NP_NN), (5, 6, NOUN_NN))], AVPNP_NN: [((3, 5, NP_NN), (5, 6, NOUN_NN)), ((3, 5, NP_NN), (5, 6, NOUN_NN))], NP_NN_VERB_VB: [((3, 5, NP_NN), (5, 6, VERB_VB))]})
chart[2][3] : defaultdict(<class 'list'>, {ADJ_OD: ['first'], ADV_RB: ['first'], first: ['first', 'first']})
chart[3][6] : defaultdict(<class 'list'>, {NOUN_NN_PP_NN: [((3, 4, NOUN_NN), (4, 6, PP_NN))], NP_NN: [((3, 4, NOUN_NN), (4, 6, PP_NN)), ((3, 4, NOUN_NN), (4, 6, PP_NN)), ((3, 5, NP_NN), (5, 6, NOUN_NN)), ((3, 5, NP_NN), (5, 6, NOUN_NN))], VERB_VB_PP_NN: [((3, 4, VERB_VB), (4, 6, PP_NN))], VP_VB: [((3, 4, VERB_VB), (4, 6, PP_NN)), ((3, 4, V

chart[5][6] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][5] : defaultdict(<class 'list'>, {ADJ_JJ: ['on'], PREP_IN: ['on'], on: ['on', 'on']})
chart[5][6] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][5] : defaultdict(<class 'list'>, {ADJ_JJ: ['on'], PREP_IN: ['on'], on: ['on', 'on']})
chart[5][6] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][5] : defaultdict(<class 'list'>, {ADJ_JJ: ['on'], PREP_IN: ['on'], on: ['on', 'on']})
chart[5][6] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][5] : defaultdict(<class 'list'>, {ADJ_JJ: ['on'], PREP_IN: ['on'], on: ['on', 'on']})
chart[5][6] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][5] : defaultdict(<class 'list'>, {ADJ_JJ: ['on'], PREP_IN: ['on'], on: ['on', 'on

chart[3][4] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][6] : defaultdict(<class 'list'>, {NP_NNS_NOUN_NN: [((4, 5, NP_NNS), (5, 6, NOUN_NN))], NP_NN: [((4, 5, NP_NNS), (5, 6, NOUN_NN)), ((4, 5, NP_NNS), (5, 6, NOUN_NN))], ADJ_JJ: [((4, 5, one), (5, 6, way)), ((4, 5, one), (5, 6, way))]})
chart[3][4] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][6] : defaultdict(<class 'list'>, {NP_NNS_NOUN_NN: [((4, 5, NP_NNS), (5, 6, NOUN_NN))], NP_NN: [((4, 5, NP_NNS), (5, 6, NOUN_NN)), ((4, 5, NP_NNS), (5, 6, NOUN_NN))], ADJ_JJ: [((4, 5, one), (5, 6, way)), ((4, 5, one), (5, 6, way))]})
chart[3][4] : defaultdict(<class 'list'>, {ADJ_AT: ['a'], NOUN_NP: ['a'], PREP_IN: ['a'], a: ['a', 'a']})
chart[4][6] : defaultdict(<class 'list'>, {NP_NNS_NOUN_NN: [((4, 5, NP_NNS), (5, 6, NOUN_NN))], NP_NN: [((4, 5, NP_NNS), (5, 6, NOUN_NN)), ((4, 5, NP_NNS), (5, 6, NOUN_NN))], ADJ_JJ: [((4, 5, one), 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [138]:
import os
os.getcwd()

'/home/rrichajalota/data/Masters/UdS/compLing/WinterSem-2020-21/Computational-Linguistics/CKY-Parser'

In [61]:
s = gramma.productions(lhs=Nonterminal('SIGMA'))
for r in s:
    print(r.rhs())
    print(gramma.productions(lhs=r.rhs()[0]))
    break

(NREL_VBZ,)
[NREL_VBZ -> NP_WPS VERB_VBZ INFCL_VB, NREL_VBZ -> NP_NN VERB_VBZ PP_NN AVP_RB NP_CD, NREL_VBZ -> NP_NN NP_WPS VERB_VBZ PP_NN, NREL_VBZ -> AVP_RB NP_PPS VERB_VBZ NP_NN, NREL_VBZ -> NP_CD VERB_VBZ]


In [78]:
for prod in gram.productions(lhs=Nonterminal('SIGMA')):
    print(prod.rhs()[0])

ADJ_ABL JYL
ADJ_ABL NOUN_CD
ADJ_ABL NOUN_NNS
ADJ_AP JYM
ADJ_AP JYN
ADJ_AP JYO
ADJ_AP NOUN_NNS
ADJ_AT ACO
ADJ_AT ACT
ADJ_AT ACY
ADJ_AT ADD
ADJ_AT ADJ_JJ
ADJ_AT ADJ_JJT
ADJ_AT ADX
ADJ_AT AEC
ADJ_AT AEH
ADJ_AT AEM
ADJ_AT AER
ADJ_AT CAA
ADJ_AT CAI
ADJ_AT CAM
ADJ_AT CAQ
ADJ_AT CAU
ADJ_AT CAY
ADJ_AT CBC
ADJ_AT CBG
ADJ_AT CBK
ADJ_AT CBO
ADJ_AT CBS
ADJ_AT CBW
ADJ_AT CCA
ADJ_AT CCE
ADJ_AT CCI
ADJ_AT CDC
ADJ_AT CDG
ADJ_AT CDK
ADJ_AT CDO
ADJ_AT CDS
ADJ_AT CDW
ADJ_AT CEA
ADJ_AT CEE
ADJ_AT CEI
ADJ_AT CEM
ADJ_AT CEQ
ADJ_AT CEU
ADJ_AT CEY
ADJ_AT EKY
ADJ_AT ELB
ADJ_AT ELE
ADJ_AT ELH
ADJ_AT ELK
ADJ_AT ELN
ADJ_AT ELQ
ADJ_AT ELT
ADJ_AT ELW
ADJ_AT ELZ
ADJ_AT EMC
ADJ_AT EMF
ADJ_AT EMI
ADJ_AT EML
ADJ_AT EMO
ADJ_AT EMR
ADJ_AT EMU
ADJ_AT EMX
ADJ_AT ENA
ADJ_AT END
ADJ_AT ENG
ADJ_AT ENJ
ADJ_AT ENM
ADJ_AT ENP
ADJ_AT ENS
ADJ_AT ENV
ADJ_AT ENY
ADJ_AT EOB
ADJ_AT EOE
ADJ_AT EOH
ADJ_AT EOK
ADJ_AT EON
ADJ_AT EOQ
ADJ_AT EOT
ADJ_AT EOW
ADJ_AT EOZ
ADJ_AT EPC
ADJ_AT EPF
ADJ_AT EPI
ADJ_AT EPL
ADJ_AT EPO
ADJ_AT EPR
ADJ_AT E

coach
month
reserve
whats
p.m
all
louis
columbus
toronto
weight
either
september
we
las
petersburg
transportation
f
listing
stopping
leg
limousine
wednesday
fuck
airfares
hub
airports
limit
these
offices
machine
diego
areas
tennessee
alaska


In [60]:
gram.productions(lhs=Nonterminal('SIGMA')) == gramma.productions(lhs=Nonterminal('SIGMA'))

False

In [91]:
for idx, sent in enumerate(test_sents):
    #print(sent[0])
    print(cky_recognizer(sent[0], gram, True))
    if idx == 5:
        break

2
number of trees: 2
2
3
number of trees: 6
6
4
number of trees: 4
4
4
number of trees: 6
6
4
number of trees: 6
6
4
number of trees: 30
30


In [45]:
grammar.productions(lhs=Nonterminal('SIGMA'))

[SIGMA -> ADJ_ABL JYL,
 SIGMA -> ADJ_ABL NOUN_CD,
 SIGMA -> ADJ_ABL NOUN_NNS,
 SIGMA -> ADJ_AP JYM,
 SIGMA -> ADJ_AP JYN,
 SIGMA -> ADJ_AP JYO,
 SIGMA -> ADJ_AP NOUN_NNS,
 SIGMA -> ADJ_AT ACO,
 SIGMA -> ADJ_AT ACT,
 SIGMA -> ADJ_AT ACY,
 SIGMA -> ADJ_AT ADD,
 SIGMA -> ADJ_AT ADJ_JJ,
 SIGMA -> ADJ_AT ADJ_JJT,
 SIGMA -> ADJ_AT ADX,
 SIGMA -> ADJ_AT AEC,
 SIGMA -> ADJ_AT AEH,
 SIGMA -> ADJ_AT AEM,
 SIGMA -> ADJ_AT AER,
 SIGMA -> ADJ_AT CAA,
 SIGMA -> ADJ_AT CAI,
 SIGMA -> ADJ_AT CAM,
 SIGMA -> ADJ_AT CAQ,
 SIGMA -> ADJ_AT CAU,
 SIGMA -> ADJ_AT CAY,
 SIGMA -> ADJ_AT CBC,
 SIGMA -> ADJ_AT CBG,
 SIGMA -> ADJ_AT CBK,
 SIGMA -> ADJ_AT CBO,
 SIGMA -> ADJ_AT CBS,
 SIGMA -> ADJ_AT CBW,
 SIGMA -> ADJ_AT CCA,
 SIGMA -> ADJ_AT CCE,
 SIGMA -> ADJ_AT CCI,
 SIGMA -> ADJ_AT CDC,
 SIGMA -> ADJ_AT CDG,
 SIGMA -> ADJ_AT CDK,
 SIGMA -> ADJ_AT CDO,
 SIGMA -> ADJ_AT CDS,
 SIGMA -> ADJ_AT CDW,
 SIGMA -> ADJ_AT CEA,
 SIGMA -> ADJ_AT CEE,
 SIGMA -> ADJ_AT CEI,
 SIGMA -> ADJ_AT CEM,
 SIGMA -> ADJ_AT CEQ,
 SIGMA -

In [92]:
gram.productions(rhs=Nonterminal('milwaukee'))

[NAPPOS_NP -> milwaukee orlando,
 NOUN_NP -> milwaukee orlando,
 AVPNP_NP -> milwaukee orlando,
 NP_NP -> milwaukee orlando,
 SIGMA -> milwaukee orlando]

In [93]:
gramma.productions(rhs=Nonterminal('milwaukee'))

[NOUN_NP -> milwaukee orlando, NOUN_NP -> milwaukee]

In [94]:
grammar.productions(rhs=Nonterminal('milwaukee'))

[NAPPOS_NP -> milwaukee orlando,
 NOUN_NP -> milwaukee orlando,
 AVPNP_NP -> milwaukee orlando,
 NP_NP -> milwaukee orlando,
 SIGMA -> milwaukee orlando]